In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_arrivals = pd.read_csv(f'./data/df_arrivals.csv',sep = ",")
df_departures = pd.read_csv(f'./data/df_departures.csv',sep = ",")

# Compagnies aériennes

## Comparaison des aéroports intra et inter

### Comparaison des vols nationaux et internationaux

In [ ]:
Airline_count = len(df_arrivals['Airline Name'])
Airline_nan_count = df_arrivals['Airline Name'].isna().sum()
print(Airline_count)
print(77567 - Airline_nan_count)

In [ ]:
df_arrivals.head(5)

In [ ]:
# Annoter les vols internationaux et nationaux pour df_arrivals
flight_type = []
for row_arr in df_departures.iterrows():
    if row_arr[1]["Country"] == row_arr[1]["Destination Country"]:
        flight_type.append("National")
    elif row_arr[1]["Country"] != row_arr[1]["Destination Country"]:
        flight_type.append("International")

df_departures["Flight Type"] = flight_type

In [ ]:
# Annoter les vols internationaux et nationaux pour df_arrivals
flight_type = []
for row_arr in df_arrivals.iterrows():
    if row_arr[1]["Country"] == row_arr[1]["Departure Country"]:
        flight_type.append("National")
    elif row_arr[1]["Country"] != row_arr[1]["Departure Country"]:
        flight_type.append("International")

df_arrivals["Flight Type"] = flight_type

In [ ]:
df_arrivals

In [ ]:
# Créer le graphique avec Seaborn
plt.figure(figsize=(12, 6))
sns.countplot(data=df_arrivals, x='Flight Type', palette='coolwarm')
plt.xlabel('Flight Category')
plt.ylabel('Number of Flights')
plt.title('Distribution of National and International Flights')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Pays et aéroport avec le plus de vols nationaux et celui avec le plus de vols internationaux

In [ ]:
#Couple pays et aéroport qui reviennent le plus souvent
# Grouper les données par aéroport et par pays, puis compter les vols nationaux et internationaux
airport_counts = df_arrivals.groupby(['Airport', 'Country', 'Flight Type']).size().reset_index(name='Flight Count')

# Identifier l'aéroport et le pays avec le plus grand nombre de vols internationaux et nationaux
airport_most_international = airport_counts[airport_counts['Flight Type'] == 'International'].max()['Airport']
country_most_international = airport_counts[airport_counts['Flight Type'] == 'International'].max()['Country']
airport_most_national = airport_counts[airport_counts['Flight Type'] == 'National'].max()['Airport']
country_most_national = airport_counts[airport_counts['Flight Type'] == 'National'].max()['Country']

print(f"Airport with most international flights: {airport_most_international},{country_most_international}")
print(f"Airport with most international flights: {airport_most_national},{country_most_national}")

### Top 5 destination les plus convoitées (qui revient le plus)

In [ ]:
df_departures.head(5)

In [ ]:
# Grouper les données par destination de vol et compter les occurrences
destination_counts = df_departures['Destination Country'].value_counts()

# Sélectionner les cinq premiers
top_5_destination = destination_counts.head(5)
top_5_destination.index.name = None
# Créer un DataFrame vide avec les destinations comme colonnes
top_countries = pd.DataFrame(columns=top_5_destination.index)
print(f"Top 4 destinations {list(top_5_destination.index)}")

# Ensuite, pour chaque destination, trouver le pays d'origine le plus fréquent
for destination_country in top_5_destination.index:
    flights_to_destination = df_departures[df_departures['Destination Country'] == destination_country]
    origin_counts = flights_to_destination['Country'].value_counts().head(4)
    top_countries[destination_country] = origin_counts.index
    print(f"Top 4 common origin country for flights to \"{destination_country}\": {list(origin_counts.index)}")

In [ ]:
top_countries
#Le premier pays d'origine (d'où proviennent les vols) sont les pays eux mêmes vu qu'il y a également les vols nationaux.

### Pays et aéroport avec le plus de vol

In [ ]:
#Ici je recherche le couple pays + aéroport qui revient le plus souvent
# Grouper les données par aéroport et par pays, puis compter le nombre total de vols
airport_counts = df_arrivals.groupby(['Airport', 'Country']).size().reset_index(name='Flight Count')

# Identifier l'aéroport et le pays avec le plus grand nombre total de vols
airport_most_flights = airport_counts.loc[airport_counts['Flight Count'].idxmax()]

print("Airport with most flights:", airport_most_flights['Airport'])
print("Country with most flights:", airport_most_flights['Country'])

### Pays avec le plus de vol

In [ ]:
#Pays avec le plus de vol à l'international
international_flights = df_arrivals[df_arrivals['Flight Type'] == "International"]
most_flights_country = international_flights['Departure Country'].value_counts().idxmax()
print(f"Le pays avec le plus de vol (Internationaux): {most_flights_country}")

#Destination favorite de ce pays (Vers où il va) 
flights_from_most_flights_country = international_flights[international_flights['Departure Country'] == most_flights_country]["Country"].value_counts()
print(f"Les destinations favorites de {most_flights_country}: {list(flights_from_most_flights_country.head(5).index)}")

### Destination favorite

In [ ]:
# Trouver la destination favorite en général
international_flights = df_departures[df_departures['Flight Type'] == "International"]
most_popular_destination = international_flights['Destination Country'].value_counts().idxmax()
print(f"La destionation favorite: {most_popular_destination}")

#Qui vient visiter
flights_to_most_popular_destination = international_flights[international_flights['Destination Country'] == most_popular_destination]["Country"].value_counts()
print(f"Les pays qui viennent le plus visiter {most_popular_destination}: {list(flights_from_most_flights_country.head(5).index)}")

# Analyse des retards

In [ ]:
df_flights = pd.read_csv(f'./data/data_usa/flights.csv', low_memory=False)

In [ ]:
print(df_flights.shape)

Vu la taille du dataset (contient tous les vols de l'année 2015), on ne considère qu'une partie en se limitant au mois d'avril

In [ ]:
df_flights_april = df_flights[df_flights['MONTH'] == 4]

In [ ]:
df_flights_april

## Pre-processing

### Dates
Dans le dataset, on a 4 colonnes qui font référence à la date, [YEAR,MONTH,DAY,DAY_OF_WEEK], on va utiliser le format datatime pour n'avoir qu'une seule colonne

In [ ]:
df_flights_april.insert(loc = 0, column = 'DATE',value = pd.to_datetime(df_flights_april.loc[:, ['YEAR','MONTH', 'DAY']]))

In [ ]:
df_flights_april

### Heures
Les heures de SCHEDULED_DEPARTURES par exemple sont mal formatées, c'est un int dont les deux premiers chiffres définissent les heures et les deux derniers les minutes.

In [ ]:
import datetime
import numpy as np
#Fonction qui convertit ce format en un format datetime.time
def convert_time(chaine):
    float(chaine.iloc[0])
    if pd.isna(chaine.iloc[0]):
        return np.nan
    else:
        if chaine.iloc[0] == 2400: 
            chaine.iloc[0] = 0
        chaine = "{0:04d}".format(int(chaine.iloc[0]))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

def format_col(df,col):
    liste = []
    for index,value in df[[col]].iterrows():
        liste.append(convert_time(value))
    return liste

In [ ]:
#Convertir les heures au bon format
df_flights_april.loc[:,'SCHEDULED_DEPARTURE']= format_col(df_flights_april, 'SCHEDULED_DEPARTURE')
df_flights_april.loc[:,'DEPARTURE_TIME'] = format_col(df_flights_april, 'DEPARTURE_TIME')
df_flights_april.loc[:,'SCHEDULED_ARRIVAL'] = format_col(df_flights_april, 'SCHEDULED_ARRIVAL')
df_flights_april.loc[:,'ARRIVAL_TIME'] = format_col(df_flights_april, 'ARRIVAL_TIME')

In [ ]:
df_flights_april[["DATE","SCHEDULED_DEPARTURE","SCHEDULED_ARRIVAL","DEPARTURE_TIME","ARRIVAL_TIME"]]

In [ ]:
df_april = df_flights_april.copy(deep=True)

### Suppression des colonnes non utilisées

In [ ]:
df_april.columns

In [ ]:
non_used_columns = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK','FLIGHT_NUMBER', 'TAIL_NUMBER','TAXI_OUT',
       'WHEELS_OFF','WHEELS_ON', 'TAXI_IN','AIR_TIME','DIVERTED', 'CANCELLED', 'CANCELLATION_REASON','AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
       'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df_april = df_flights_april.drop(non_used_columns,axis=1)
df_april

In [ ]:
#Convertir les distances de miles à kilomètres
import math
def miles_to_kilometers(miles):
    return math.ceil(float(miles.iloc[0]) * 1.60934)

def convert_distances(df,col):
    liste = []
    for index,value in df[[col]].iterrows():
        liste.append(miles_to_kilometers(value))
    return liste

In [ ]:
df_april.loc[:,'DISTANCE']= convert_distances(df_april, 'DISTANCE')

In [ ]:
df_april

## Analyse

### Valeurs manquantes

In [ ]:
missing_df = df_april.isnull().sum(axis=0).reset_index()
missing_df.columns = ['Colonne', 'NaN Values']
missing_df['Filling factor (%)']=(df_april.shape[0]-missing_df['NaN Values'])/df_april.shape[0]*100
missing_df.sort_values('Filling factor (%)').reset_index(drop = True)

On voit bien que la majorité des colonnes sont pleines et celles qui sont vides ont un filling factor très élevée (le dataset est bien rempli).

In [ ]:
df_april.describe()

On se concentre sur les colonnes DEPARTURE_DELAY et ARRIVAL_DELAY, la moyenne dans tous le dataset est respectivement de: 7.721885 et 3.163190 min.
Pourtant 50% des vols décollent avant l'heure prévue d'environ 2 min et arrivent avant l'heure prévue de 5 min. 

Mais à 75%, on note un retard d'environ 6 et 7 min au décollage et à l'arrivée.

Le plus grand retard enregistrée est d'environ 24,9 heures au décollage et de 25,9 heures à l'arrivée.

### Airlines

#### Departure Delay par compagnies aériennes

In [ ]:
def get_stats(group):
    return {'count': group.count(),'min': group.min(), 'max': group.max(), 'mean': group.mean()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
delay_per_airline_stats = df_april['DEPARTURE_DELAY'].groupby(df_april['AIRLINE']).apply(get_stats).unstack()
delay_per_airline_stats = delay_per_airline_stats.sort_values('count')
delay_per_airline_stats

La compagnie aérienne qui fait le plus de retard environ 12 min au décollage est la compagnie aérienne UA: United Air Lines Inc.

Celle qui fait le moins de retard avec une moyenne d'environ -2 min est HA: Hawaiian Airlines Inc.

Le pire retard enregistré (25.7) a été effectué par la compagnie AA : American Airlines Inc.

La meilleure avance enregistré (68 min) a été effectué par la compagnie AA aussi (American Airlines Inc.)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Iterate through each airline
for airline in delay_per_airline_stats.index:
    # Filter data for the current airline
    data = df_april[df_april['AIRLINE'] == airline]
    
    # Create a violin plot for the current airline
    plt.figure(figsize=(6, 4))
    sns.violinplot(x = "AIRLINE",y="DEPARTURE_DELAY",data=data)

    # Add title and labels
    plt.title(f'Distribution of Departure Delays for {airline}')
    plt.xlabel('Departure Delay (minutes)')
    plt.ylabel('Density')
    
    # Show the plot
    plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Iterate through each airline
for airline in delay_per_airline_stats.index:
    # Filter data for the current airline
    data = df_april[df_april['AIRLINE'] == airline]
    
    sns.boxplot(x = "DEPARTURE_DELAY",data = data)
    plt.title('Box Plot of Departure Delays by Airline')
    plt.xlabel('Airline')
    plt.ylabel('Departure Delay (minutes)')
    plt.xticks(rotation=45)  # Rotate labels for better readability
    plt.show()
    
    # Show the plot
    plt.show()
# Créer un box plot avec Seaborn
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_april['AIRLINE'], y=df_april['DEPARTURE_DELAY'])
plt.title('Box Plot of Departure Delays by Airline')
plt.xlabel('Airline')
plt.ylabel('Departure Delay (minutes)')
plt.xticks(rotation=45)  # Rotate labels for better readability
plt.show()

In [ ]:
# I redefine the colors for correspondance with the pie charts
colors = ['firebrick', 'gold', 'lightcoral', 'aquamarine', 'c', 'yellowgreen', 'grey',
          'seagreen', 'tomato', 'violet', 'wheat', 'chartreuse', 'lightskyblue', 'royalblue']
plt.figure(figsize=(10, 6))
sns.stripplot(y="AIRLINE", x="DEPARTURE_DELAY", size = 4, palette = colors,
                    data=df_april, linewidth = 0.5,  jitter=True)
plt.show()

La plupart des retards de départ pour toutes les compagnies aériennes sont compris entre environ -7 minutes et 20 minutes. Cependant, pour certaines compagnies, ces retards peuvent aller jusqu'à environ 50 minutes. Au-delà de ces durées habituelles, il existe des durées bien plus élevées, mais ces valeurs sont considérées comme des données aberrantes, car elles surviennent rarement.

#### % de retard (moyennes) par compagnies

In [ ]:
# Extract labels and sizes from delay_per_airline_stats DataFrame
labels = delay_per_airline_stats.index
sizes = delay_per_airline_stats['mean'].values
sizes  = [max(s,0) for s in sizes]

# Plot the pie chart
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, autopct=lambda x: f"{x:.0f}", startangle=0)

# Set title and aspect ratio
plt.title('% of mean delay per airline', fontsize=18)

# Show the plot
plt.show()

#### % de vols par compagnies

In [ ]:
print(delay_per_airline_stats.index)

In [ ]:
# Extract labels and sizes from delay_per_airline_stats DataFrame
labels = delay_per_airline_stats.index
sizes = delay_per_airline_stats['count'].values

# Set explode values based on sizes
explode = [0.3 if size < 20000 else 0.0 for size in sizes]

# Plot the pie chart
plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, autopct='%1.0f%%', startangle=0)

# Set title and aspect ratio
plt.title('% of flights per airline', fontsize=18)
plt.axis('equal')

# Show the plot
plt.show()

La compagnie aérienne WN (Southwest Airlines Co.) est la compagnie aérienne qui effectue le plus de vol pourtant ce n'est pas celle qui commet la plus grande moyenne de crimes (elle est classée 5ème parmi toutes les compagnies).

Toutefois le premier camembert montre qu'il y a très peut de différences entre les compagnies aériennes pour les retards commis, à l'exception des compagnies Hawaiian Airlines et Alaska Airlines qui ont une moyenne proche de 0, le reste des compagnies ont une moyenne qui reste relativement faible entre 7 et 13 min.

#### Comparaison entre le retard effectuée au départ et à l'arrivée

In [ ]:
fig = plt.figure(1, figsize=(11,6))
ax = sns.barplot(x="DEPARTURE_DELAY", y="AIRLINE", data=df_april, color="lightblue", errorbar=None)
ax = sns.barplot(x="ARRIVAL_DELAY", y="AIRLINE", data=df_april, hatch = '///',alpha = 0.0, errorbar=None)
plt.xlabel('Mean delay [min] (@departure: blue, @arrival: hatch lines)',fontsize=14, weight = 'bold', labelpad=10);

On remarque que pour la majorité des compagnies aériennes, le retard qu'elles ont effectuée au départ est rattrapée lors du vol. Elles ajustent la durée du vol afin qu'il n'y ait pas de conséquences sur l'heure d'arrivée, sauf pour les compagnies AS:Alaska Airlines Inc. et HA:Hawaiian Airlines Inc. qui respectivement arrivent en avance et décollent avant l'heure prévue

### Relation between distance and delays

In [ ]:
# Calcul de la matrice de corrélation
correlation_matrix = df_april[['DISTANCE', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']].corr()

# Affichage de la heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Between Distance and Delays')
plt.show()

La distance n'a pas d'incidence sur les retards, mais évidemment il y a une forte corrélation entre le retard au décollage et à l'arrivée, on a tout de même vu plus haut que le retard à l'arrivée était rattrapée par rapport au décollage

### Aéroports et Retards

In [ ]:
airlines_names = pd.read_csv('data/data_usa/airlines.csv')
airports = pd.read_csv('data/data_usa/airports.csv')

In [ ]:
abbr_companies = airlines_names.set_index('IATA_CODE')['AIRLINE'].to_dict()

In [ ]:
identify_airport = airports.set_index('IATA_CODE')['CITY'].to_dict()

In [ ]:
pd.set_option('display.max_rows', df_april.shape[0]+1)

#### Statistiques des aéroports

In [ ]:
def get_stats(group):
    return {'count': group.count(),'min': group.min(), 'max': group.max(), 'mean': group.mean()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
delay_per_airport_stats = df_april['DEPARTURE_DELAY'].groupby(df_april['ORIGIN_AIRPORT']).apply(get_stats).unstack()
delay_per_airport_stats = delay_per_airport_stats.sort_values('count')
styled_per_airport_stats = delay_per_airport_stats.style.highlight_max(color='lightgreen').highlight_min(color='lightcoral')
# Display the styled DataFrame
print(delay_per_airport_stats["mean"].mean())

In [ ]:
styled_per_airport_stats

Le retard moyen au décollage enregistré pour le mois d'avril entre tous les aéroports est d'environ 5 min ce qui est considéré comme un retard  acceptable. Le pire retard enregistré a été effectué par l'aéroport de San Diego avec 25,7 heures de retard mais en moyenne ce n'est pas l'aéroport qui fait le plus de retard (6.78 min). En moyenne c'est l'aéroport de Jack Brooks Regional qui est le plus en retard (31,78 min) mais on peut pas encore le classer comme étant le pire aéroport, ça dépend du nombre de vols qu'ils effectuent (si cet aéroport n'effectue pas beaucoup de vols c'est normal que sa moyenne soit élevée).

#### Statistique des aéroports en fonction des compagnies aériennes

##### Au départ

In [ ]:
delay_per_airport_airline_stats = df_april['DEPARTURE_DELAY'].groupby([df_april['AIRLINE'], df_april['ORIGIN_AIRPORT']]).apply(get_stats).unstack()
delay_per_airport_airline_stats = delay_per_airport_airline_stats.sort_values(by='count').sort_index()

# Display the styled DataFrame
print(delay_per_airport_airline_stats["mean"].mean())

In [ ]:
# Réorganiser les niveaux de l'index pour avoir les compagnies aériennes en premier niveau
delay_per_airport_airline_stats = delay_per_airport_airline_stats.swaplevel(1,0);

# Trier les index par compagnies aériennes
delay_per_airport_airline_stats = delay_per_airport_airline_stats.sort_index(level=0)

level_names = delay_per_airport_airline_stats.index.names

# Define a function to apply the styling for each airline
def highlight_best_worst_per_airline(s):
    max_val = s.groupby(level=level_names.index('AIRLINE')).max()
    min_val = s.groupby(level=level_names.index('AIRLINE')).min()
    return ['background-color: lightgreen' if v == max_val[a] else 'background-color: lightcoral' if v == min_val[a] else '' for a, v in zip(s.index.get_level_values(level_names.index('AIRLINE')), s)]

# Apply the styling function on each airline
styled_per_airport_airline_stats = delay_per_airport_airline_stats.style.apply(highlight_best_worst_per_airline, axis=0)

# Display the DataFrame with the style
styled_per_airport_airline_stats

En moyenne, les pires et meilleures délais on été effectué à partir des aéroports suivant pour chaque compagnie aériennes:

AA:

    - Meilleur: JAC avec environ 13,86 minutes d'avance au décollage
    - Pire: SDF avec environ 39,77 minutes de retard

AS:

    - Meilleur: LIH avec environ 8,04 minutes d'avance au décollage
    - Pire: TPA avec environ 11,46 minutes de retard


B6:

    - Meilleur: DTW avec environ 0,84 minutes d'avance au décollage
    - Pire: SJC avec environ 33,83 minutes de retard

DL:

    - Meilleur: ONT avec environ 6,5 minutes d'avance au décollage
    - Pire: PWM avec environ 25,20 minutes de retard

EV:

    - Meilleur: BIL avec environ 9,30 minutes d'avance au décollage
    - Pire: HOU avec environ 37,73 minutes de retard

F9:

    - Meilleur: BZN avec environ 9 minutes d'avance au décollage
    - Pire: BMI avec environ 35 minutes de retard

HA:

    - Meilleur: JFK avec environ 9,04 minutes d'avance au décollage
    - Pire: LAS avec environ 12,06 minutes de retard

MQ:

    - Meilleur: LAX avec environ 5,40 minutes d'avance au décollage
    - Pire: SAF avec environ 35,5 minutes de retard

NK:

    - Meilleur: PBI avec environ 0,9 minutes de retard au décollage
    - Pire: MSY avec environ 18 minutes de retard

OO:

    - Meilleur: GUC avec environ 14 minutes d'avance au décollage
    - Pire: CHO avec environ 25,76 minutes de retard

UA:

    - Meilleur: HDN avec environ 7,38 minutes d'avance au décollage
    - Pire: BQN avec environ 74,64 minutes de retard

US:

    - Meilleur: GEG avec environ 7,03 minutes d'avance au décollage
    - Pire: ORF avec environ 37 minutes de retard

VX:

    - Meilleur: DCA avec environ 2,58 minutes d'avance au décollage
    - Pire: MCO avec environ 23,73 minutes de retard

WN:

    - Meilleur: DSM avec environ 0,73 minutes d'avance au décollage
    - Pire: SDF avec environ 14,09 minutes de retard


A première vue, le meilleur aéroport en termes de temps de retard serait l'aéroport GUC et le pire serait l'aéroport BQN

##### Afficher les 10 pires aéroports et les 10 meilleurs

In [ ]:
# Trier les aéroports par retard moyen
sorted_mean_delay = delay_per_airport_stats.sort_values(by='mean')
# Calculate normalized mean delay for each airport
normalized_mean_delay = sorted_mean_delay['mean'] / sorted_mean_delay['count']

# Set the number of airports to display on each end (e.g., top 10 and bottom 10)
top_n = 5
bottom_n = 5

# Select the top and bottom airports based on normalized mean delay
top_airports = normalized_mean_delay.tail(top_n)
bottom_airports = normalized_mean_delay.head(bottom_n)

# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Plot the normalized mean delays for top airports
axes[0].bar(top_airports.index, top_airports, color='skyblue')
axes[0].set_title('Top Airports by Normalized Mean Delay')
axes[0].set_ylabel('Normalized Mean Delay')
axes[0].tick_params(axis='x', rotation=45, labelsize=8)  # Rotate x-axis labels for better visibility

# Plot the normalized mean delays for bottom airports
axes[1].bar(bottom_airports.index, bottom_airports, color='skyblue')
axes[1].set_title('Bottom Airports by Normalized Mean Delay')
axes[1].set_ylabel('Normalized Mean Delay')
axes[1].tick_params(axis='x', rotation=45, labelsize=8)  # Rotate x-axis labels for better visibility

plt.tight_layout()
plt.show()

Après normalisation des valeurs (vu que le nombre de vols effectués par chaque comagnie aérienne à partir de ces aéroports n'est pas le même le résultat peut être biaisé) le graphe confirme en effet que GUC présente l'une des meilleurs performances après l'aéroport HDN (ne fait pas de retard en moyenne et est en avance) et BQN fait parti en effet des trois pires aéroports après les aéroports MQT et GUN.

##### A l'arrivée

In [ ]:
arrival_per_airport_airline_stats = df_april['ARRIVAL_DELAY'].groupby([df_april['AIRLINE'], df_april['ORIGIN_AIRPORT']]).apply(get_stats).unstack()
arrival_per_airport_airline_stats = arrival_per_airport_airline_stats.sort_values(by='count').sort_index()

# Display the styled DataFrame
print(arrival_per_airport_airline_stats["mean"].mean())

In [ ]:
# Réorganiser les niveaux de l'index pour avoir les compagnies aériennes en premier niveau
arrival_per_airport_airline_stats = arrival_per_airport_airline_stats.swaplevel(1,0);

# Trier les index par compagnies aériennes
arrival_per_airport_airline_stats = arrival_per_airport_airline_stats.sort_index(level=0)

level_names_arrival = arrival_per_airport_airline_stats.index.names

# Define a function to apply the styling for each airline
def highlight_best_worst_per_airline_arrival(s):
    max_val = s.groupby(level=level_names_arrival.index('AIRLINE')).max()
    min_val = s.groupby(level=level_names_arrival.index('AIRLINE')).min()
    return ['background-color: lightgreen' if v == max_val[a] else 'background-color: lightcoral' if v == min_val[a] else '' for a, v in zip(s.index.get_level_values(level_names_arrival.index('AIRLINE')), s)]

# Apply the styling function on each airline
styled_per_airport_airline_stats_arrival = arrival_per_airport_airline_stats.style.apply(highlight_best_worst_per_airline_arrival, axis=0)

# Display the DataFrame with the style
styled_per_airport_airline_stats_arrival

En moyenne, les pires et meilleures délais on été effectué à partir des aéroports suivant pour chaque compagnie aériennes:

AA:

    - Meilleur: HDN avec environ 15,85 minutes d'avance à l'arrivée
    - Pire: SDF avec environ 47,08 minutes de retard

AS:

    - Meilleur: EWR avec environ 18,01 minutes d'avance à l'arrivée
    - Pire: JNU avec environ 9 minutes de retard


B6:

    - Meilleur: DTW avec environ 10,18 minutes d'avance à l'arrivée
    - Pire: SJC avec environ 32,65 minutes de retard

DL:

    - Meilleur: MHT avec environ 15,23 minutes d'avance à l'arrivée
    - Pire: ROA avec environ 20,53 minutes de retard

EV:

    - Meilleur: PHL avec environ 18,20 minutes d'avance à l'arrivée
    - Pire: HOU avec environ 32,33 minutes de retard

F9:

    - Meilleur: BZN avec environ 5,56 minutes d'avance à l'arrivée
    - Pire: SFO avec environ 31,16 minutes de retard

HA:

    - Meilleur: SEA avec environ 6,51 minutes d'avance à l'arrivée
    - Pire: LAS avec environ 20,91 minutes de retard

MQ:

    - Meilleur: LAX avec environ 9,60 minutes d'avance à l'arrivée
    - Pire: SAF avec environ 40,63 minutes de retard

NK:

    - Meilleur: SJU avec environ 4,77 minutes de retard à l'arrivée
    - Pire: MSY avec environ 22.95 minutes de retard

OO:

    - Meilleur: GUC avec environ 25,50 minutes de retard à l'arrivée
    - Pire: CHS avec environ 24,01 minutes de retard

UA:

    - Meilleur: HDN avec environ 21,75 minutes d'avance à l'arrivée
    - Pire: BQN avec environ 70,32 minutes de retard

US:

    - Meilleur: KOA avec environ 23,31 minutes d'avance à l'arrivée
    - Pire: ORF avec environ 51,25 minutes de retard

VX:

    - Meilleur: DCA avec environ 11,75 minutes d'avance à l'arrivée
    - Pire: MCO avec environ 18,13 minutes de retard

WN:

    - Meilleur: PWM avec environ 5,88 minutes d'avance à l'arrivée
    - Pire: LAX avec environ 7,69 minutes de retard


A première vue, le meilleur aéroport en termes de temps de retard serait l'aéroport GUC et le pire serait l'aéroport BQN toujours (ce qui est logique car BQN a un retard moyen d'environ 74 minutes au décollage donc il a du mal à rattraper ce retard à l'arrivée et GUC décolle en moyenne 14 minutes en avance).

In [ ]:
# Trier les aéroports par retard moyen
sorted_mean_arrival = arrival_per_airport_airline_stats.sort_values(by='mean')
# Calculate normalized mean delay for each airport
normalized_mean_arrival = sorted_mean_arrival['mean'] / sorted_mean_arrival['count']
# Set the number of airports to display on each end (e.g., top 10 and bottom 10)
top_n = 5
bottom_n = 5

# Select the top and bottom airports based on normalized mean delay
top_airports_arrival = normalized_mean_arrival.tail(top_n)
bottom_airports_arrival = normalized_mean_arrival.head(bottom_n)
# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(12, 10))
top_airports_arrival.index = top_airports_arrival.index.map(str)
bottom_airports_arrival.index = bottom_airports_arrival.index.map(str)
# Plot the normalized mean delays for top airports
axes[0].bar(top_airports_arrival.index, top_airports_arrival, color='skyblue')
axes[0].set_title('Top Airports by Normalized Mean Delay')
axes[0].set_ylabel('Normalized Mean Delay')
axes[0].tick_params(axis='x', rotation=45, labelsize=8)  # Rotate x-axis labels for better visibility

# Plot the normalized mean delays for bottom airports
axes[1].bar(bottom_airports_arrival.index, bottom_airports_arrival, color='skyblue')
axes[1].set_title('Bottom Airports by Normalized Mean Delay')
axes[1].set_ylabel('Normalized Mean Delay')
axes[1].tick_params(axis='x', rotation=45, labelsize=8)  # Rotate x-axis labels for better visibility

plt.tight_layout()
plt.show()

Après la normalisation par contre, on remarque que BQN n'est plus le pire aéroport c'est ORF avec la compagnie aérienne US, et le meilleur reste GUC avec OO.

### Analyse des tendances temporelles

#### Preprocess du dataframe entier

In [ ]:
df_flights.insert(loc = 0, column = 'DATE',value = pd.to_datetime(df_flights.loc[:, ['YEAR','MONTH', 'DAY']]))

In [ ]:
df_flights.loc[:,'SCHEDULED_DEPARTURE']= format_col(df_flights, 'SCHEDULED_DEPARTURE')
df_flights.loc[:,'DEPARTURE_TIME'] = format_col(df_flights, 'DEPARTURE_TIME')
df_flights.loc[:,'SCHEDULED_ARRIVAL'] = format_col(df_flights, 'SCHEDULED_ARRIVAL')
df_flights.loc[:,'ARRIVAL_TIME'] = format_col(df_flights, 'ARRIVAL_TIME')

In [ ]:
df_flights[["DATE","SCHEDULED_DEPARTURE","SCHEDULED_ARRIVAL","DEPARTURE_TIME","ARRIVAL_TIME"]]

In [ ]:
#Drop non relevant columns
non_used_columns = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK','FLIGHT_NUMBER', 'TAIL_NUMBER','TAXI_OUT',
       'WHEELS_OFF','WHEELS_ON', 'TAXI_IN','AIR_TIME','DIVERTED', 'CANCELLED', 'CANCELLATION_REASON','AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
       'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df_flights = df_flights.drop(non_used_columns,axis=1)
df_flights

In [ ]:
df_flights.loc[:,'DISTANCE']= convert_distances(df_flights, 'DISTANCE')

In [ ]:
#Missing values
missing_df = df_april.isnull().sum(axis=0).reset_index()
missing_df.columns = ['Colonne', 'NaN Values']
missing_df['Filling factor (%)']=(df_april.shape[0]-missing_df['NaN Values'])/df_april.shape[0]*100
missing_df.sort_values('Filling factor (%)').reset_index(drop = True)

In [ ]:
# Supprimer les valeurs NaN
df_flights.dropna(inplace = True)

In [ ]:
df_flights.to_csv(f'./data/data_usa/flights_final.csv',index=False)

In [ ]:
df_flights = pd.read_csv(f'./data/data_usa/flights_final.csv',low_memory=False)

#### Par jour de la semaine

In [ ]:
# Créer des colonnes pour le jour de la semaine et le mois
df_flights['DATE'] = pd.to_datetime(df_flights['DATE'])
df_flights['WEEKDAY'] = df_flights['DATE'].dt.day_name()

# Calculer la moyenne des retards par jour de la semaine
average_delay_weekday = df_flights.groupby('WEEKDAY')['DEPARTURE_DELAY'].mean()

# Visualisation
plt.figure(figsize=(10, 5))
average_delay_weekday.plot(kind='bar', color='orange')
plt.title('Retard moyen de départ par jour de la semaine')
plt.xlabel('Jour de la semaine')
plt.ylabel('Retard moyen (minutes)')
plt.show()

In [ ]:
# Création d'un graphique à barres pour visualiser le nombre de vols par mois
flights_per_weekday = df_flights.groupby('WEEKDAY').size()
# Normaliser le nombre de vols par le total pour obtenir des proportions
total_flights = flights_per_weekday.sum()
normalized_flights_per_weekday = flights_per_weekday / total_flights

# Création d'un graphique à barres pour visualiser les proportions de vols par mois
normalized_flights_per_weekday.plot(kind='bar')
plt.title('Proportion de Vols par jour')
plt.xlabel('Jour de la semaine')
plt.ylabel('Proportion de Vols')
plt.xticks(ticks=range(len(normalized_flights_per_weekday)), labels=[str(weekday) for weekday in normalized_flights_per_weekday.index], rotation=45)
plt.show()

Les jours de la semaine où il y a le plus de retards sont les jours de la semaine où il y a le plus de vols (soit le lundi)

#### Par mois de l'année

In [ ]:
# Créer des colonnes pour le jour de la semaine et le mois
df_flights['MONTH'] = df_flights['DATE'].dt.month_name()

# Calculer la moyenne des retards par jour de la semaine
average_delay_month = df_flights.groupby('MONTH')['DEPARTURE_DELAY'].mean()

# Visualisation
plt.figure(figsize=(10, 5))
average_delay_month.plot(kind='bar', color='orange')
plt.title("Retard moyen de départ par mois de l'année")
plt.xlabel("Mois de l'année")
plt.ylabel('Retard moyen (minutes)')
plt.show()

In [ ]:
# Création d'un graphique à barres pour visualiser le nombre de vols par mois
flights_per_month = df_flights.groupby('MONTH').size()
# Normaliser le nombre de vols par le total pour obtenir des proportions
total_flights = flights_per_month.sum()
normalized_flights_per_month = flights_per_month / total_flights

# Création d'un graphique à barres pour visualiser les proportions de vols par mois
normalized_flights_per_month.plot(kind='bar')
plt.title('Proportion de Vols par Mois')
plt.xlabel('Mois')
plt.ylabel('Proportion de Vols')
plt.xticks(ticks=range(len(normalized_flights_per_month)), labels=[str(month) for month in normalized_flights_per_month.index], rotation=45)
plt.show()

#### Par saisons

In [ ]:
def month_to_season(month):
    if month in ['December', 'January', 'February']:
        return 'Winter'
    elif month in ['March', 'April', 'May']:
        return 'Spring'
    elif month in ['June', 'July', 'August']:
        return 'Summer'
    elif month in ['September', 'October', 'November']:
        return 'Fall'

# Appliquer la fonction à la colonne 'MONTH' pour créer une nouvelle colonne 'SEASON'
df_flights['SEASON'] = df_flights['MONTH'].apply(month_to_season)

In [ ]:
# Calculer la moyenne des retards par saison
average_delay_season = df_flights.groupby('SEASON')['DEPARTURE_DELAY'].mean()

# Visualisation des retards moyens par saison
plt.figure(figsize=(10, 5))
average_delay_season.reindex(['Winter', 'Spring', 'Summer', 'Fall']).plot(kind='bar', color='blue')
plt.title("Retard moyen de départ par saison")
plt.xlabel("Saison")
plt.ylabel('Retard moyen (minutes)')
plt.show()

In [ ]:
# Calculer le nombre de vols par saison
flights_per_season = df_flights.groupby('SEASON').size()
# Normaliser le nombre de vols par le total pour obtenir des proportions
total_flights_per_season = flights_per_season.sum()
normalized_flights_per_season = flights_per_season / total_flights_per_season

# Création d'un graphique à barres pour visualiser les proportions de vols par mois
normalized_flights_per_season.plot(kind='bar')
plt.title('Proportion de Vols par Mois')
plt.xlabel('Mois')
plt.ylabel('Proportion de Vols')
plt.xticks(ticks=range(len(normalized_flights_per_season)), labels=[str(season) for season in normalized_flights_per_season.index], rotation=45)
plt.show()

C'est durant l'été que le plus de retards sont enregistrés (surtout le mois de juin dans l'analyse par mois) ce qui pourrait être expliqué par le nombre de vols à cette période qui est plus élevée par rapport au nombre tout au long de l'année.

La deuxième saison qui enregistre le plus de retards est l'hiver (surtout le mois de février et décembre dans l'analyse par mois), toutefois ce n'est pas la saison où le plus de vols à été enregistré après l'été mais ça peut s'expliquer par les conditions météorologiques.

### Analyse de corrélation (jour/mois et retard)

In [ ]:
df_flights['IS_DELAYED_DEPARTURE'] = df_flights['DEPARTURE_DELAY'].apply(lambda x: 1 if x > 0 else 0)  # 1 pour retardé, 0 sinon

# Création de variables indicatrices pour les jours de la semaine et le mois
correlation_data = pd.get_dummies(df_flights, columns=["MONTH", "WEEKDAY"])

# Ajouter la colonne de retard moyen par vol
correlation_data['AVERAGE_DELAY'] = df_flights['DEPARTURE_DELAY']
correlation_data = correlation_data.drop(["AIRLINE","ORIGIN_AIRPORT","DESTINATION_AIRPORT","SCHEDULED_DEPARTURE","DEPARTURE_TIME","SCHEDULED_ARRIVAL","ARRIVAL_TIME"],axis =1)
# Calcul de la matrice de corrélation
correlation_matrix = correlation_data.corr()

In [ ]:
import numpy as np
columns_to_include = [col for col in correlation_matrix.columns if 'MONTH' in col or 'WEEKDAY' in col or col in correlation_matrix.columns]
correlation_focus = correlation_matrix[columns_to_include]

# Affichage de la matrice de corrélation
plt.figure(figsize=(20, 15))
sns.heatmap(correlation_focus,annot = True, cmap='coolwarm', fmt=".2f",yticklabels=correlation_matrix.columns,annot_kws={"size": 10})
plt.title('Corrélation entre les retards, les jours et les mois')
plt.show()

Corrélations Fortes:

    - Les variables DEPARTURE_DELAY et ARRIVAL_DELAY présentent une forte corrélation (0.87), ce qui est attendu puisque les retards au départ peuvent souvent entraîner des retards à l'arrivée.

    - SCHEDULED_TIME et ELAPSED_TIME montrent également une corrélation élevée, indiquant que le temps de vol prévu est souvent proche du temps de vol réel.

Corrélations avec les mois et les jours de la semaine:

    - Les mois et les jours de la semaine montrent généralement une faible corrélation avec DEPARTURE_DELAY, à l'exception de certains mois spécifiques comme juin, décembre et février qui montrent une légère corrélation positive (jusqu'à 0.14).

    - Il y a peu de corrélation entre les jours de la semaine et DEPARTURE_DELAY, avec des valeurs allant de -0.01 à 0.03, ce qui implique que le jour de la semaine n'a pas d'influence significative sur les retards de départ (sauf pour le lundi)départ.

### Régression multiple

In [ ]:
import statsmodels.api as sm

# Préparer les données pour la régression
X = pd.get_dummies(df_flights[['MONTH', 'WEEKDAY']], drop_first=True)  # Utiliser drop_first pour éviter la multicolinéarité
X = X.astype(int)
X['DISTANCE'] = df_flights['DISTANCE']
Y = df_flights['DEPARTURE_DELAY']

# Ajouter une constante à X
X = sm.add_constant(X)

# Construire le modèle de régression
model = sm.OLS(Y, X).fit()

# Afficher le résumé du modèle
print(model.summary())

Coefficients des variables:

    - const: Le coefficient constant est de 6.5669 avec un P>|t| de 0.000, signifiant qu'en l'absence de toutes les autres variables, le retard moyen est de 6.57 minutes (statistiquement significatif)

    - Mois: Les mois de février, juin et décembre montrent des coefficient positifs significatifs, indiquant des retards plus élevés pendant ces mois. Les mois de mai, septembre et octobre montrent des coefficients négatifs, suggérant moins de retard

    - Jour de la semaine: Aucun des jours de la semaine n'est significativement lié à une augmentation des retards, sauf le lundi qui a un petit effet positif

    - Distante: le coefficient pour la distance est de 0.0009 et est statistiquement significatif, indiquant que pour chaque unité supplémentaire de distance, le retard augmente en moyenne de 0.0009 minutes.9 minute.

In [ ]:
import statsmodels.api as sm

# Préparer les données pour la régression
X = pd.get_dummies(df_flights[['MONTH', 'WEEKDAY']], drop_first=True)  # Utiliser drop_first pour éviter la multicolinéarité
X = X.astype(int)
X['DISTANCE'] = df_flights['DISTANCE']
Y = df_flights['ARRIVAL_DELAY']

# Ajouter une constante à X
X = sm.add_constant(X)

# Construire le modèle de régression
model = sm.OLS(Y, X).fit()

# Afficher le résumé du modèle
print(model.summary())

Coefficients des variables:

    - const: Le retard moyen à l'arrivée est de 4.8322 minutes en l'absence d'autres factuerus (statistiquement significatif).
    
    - Mois: Les mois de février, juin montrent des coefficient positifs significatifs, indiquant des retards plus élevés pendant ces mois. Les mois de septembre et octobre montrent des coefficients négatifs, suggérant moins de retard

    - Jour de la semaine: Le lundi montre un effet positif significatif sur les retards, tandis que le dimanche et le samedi montrent des effets négatifs.

    - Distante: le coefficient est pratiquement nul (-0.0012) et statistiquement non significatif, indiquant que la distance parcourue n'influence pas significativement les retards à l'arrivée, (c'est vrai vu que les vols qui ont décollé en retard arrivent à rattraper légèrement le retard ensuite à l'arrivée quelque soit la distance).ivée.

### PCA

In [ ]:
from sklearn.preprocessing import StandardScaler

# Supposons que df_flights est votre DataFrame et contient plusieurs variables numériques
# Sélectionnez les colonnes numériques pertinentes
columns = ["DEPARTURE_DELAY","SCHEDULED_TIME","ELAPSED_TIME","DISTANCE","ARRIVAL_DELAY"]
data = df_flights[columns]

# Normalisation des données
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:
from sklearn.decomposition import PCA

# Application de la PCA
pca = PCA(n_components=2)  # Réduction à 2 dimensions pour visualisation
principal_components = pca.fit_transform(data_scaled)

# Création d'un nouveau DataFrame pour les composantes principales
principal_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

In [ ]:
# Obtenir les loadings
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# Créer un DataFrame pour les loadings
loading_matrix = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=columns)

# Trouver l'index de la valeur maximale dans chaque colonne
most_influential_names = loading_matrix.idxmax(axis=0)

principal_df_columns = [f'PC1 ({most_influential_names.iloc[0]})', f'PC2 ({most_influential_names.iloc[1]})']
principal_df = pd.DataFrame(principal_components, columns=principal_df_columns)

In [ ]:
principal_df

In [ ]:
import numpy as np
def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    for d1, d2 in axis_ranks:  # pour chaque paire d'axes
        if d2 < n_comp:
            fig, ax = plt.subplots(figsize=(7,7))

            # Limites pour les axes avec un peu de marge
            circle = plt.Circle((0, 0), 1, edgecolor='b', facecolor='none')
            ax.add_artist(circle)

            # Lignes pour le quadrillage
            plt.axhline(0, color='grey', linestyle='--')
            plt.axvline(0, color='grey', linestyle='--')

            # Ajout des flèches
            for i in range(len(pcs)):
                ax.arrow(0, 0, pcs[i, d1], pcs[i, d2], head_width=0.03, head_length=0.05, fc='red', ec='red')
                if labels is not None:
                    plt.text(pcs[i, d1] * 1.15, pcs[i, d2] * 1.15, labels[i], fontsize=10, ha='right', va='center')

            # Définir les limites des axes
            plt.xlim(-1, 1)
            plt.ylim(-1, 1)
            plt.xlabel(f'PC{d1+1} - {var_exp[d1]:.2f}%')
            plt.ylabel(f'PC{d2+1} - {var_exp[d2]:.2f}%')
            plt.title("Cercle de Corrélation des PCA")
            plt.show()

# Les variances expliquées par les composantes
var_exp = pca.explained_variance_ratio_ * 100

# Appel de la fonction pour afficher le cercle de corrélation
display_circles(pca.components_.T, 2, pca, [(0, 1)], labels=columns) # empty matrix for coordinates